In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from itertools import combinations
from scipy import stats
import random
from sklearn.preprocessing import MinMaxScaler

In [2]:
from catboost import CatBoostRegressor,CatBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor,RandomForestRegressor,StackingRegressor 
import seaborn as sns
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
train_df = pd.read_csv("train_v9rqX0R.csv")
#check the info of the dataset
print(train_df.shape) 
train_df.info()
print('-'*30)
# resource_data = pd.read_csv("/content/drive/My Drive/ Deep learning assignments/ LSTM Donors_choose/resources.csv")
# #check the info of the dataset
# resource_data .info()

(8523, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB
------------------------------


In [5]:
train_df.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [6]:
test_df= pd.read_csv("test_AbJTz2l.csv")
print(test_df.shape)
# test_df.fillna(-1,inplace=True)
test_df.info()

(5681, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                4705 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Identifier          5681 non-null   object 
 7   Outlet_Establishment_Year  5681 non-null   int64  
 8   Outlet_Size                4075 non-null   object 
 9   Outlet_Location_Type       5681 non-null   object 
 10  Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 488.3+ KB


In [7]:
num_columns=['Item_Weight','Item_Visibility','Item_MRP']
# cat_columns=['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']
cat_columns=['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']

# Null columns and values

In [14]:
nan_=[i for i in train_df.columns if train_df[i].isnull().any()]
nan_

['Item_Weight', 'Outlet_Size']

There are two features that have missing values one is categorical(Outlet_Size) and other is continuous(Item_Weight) they both need to be imputed with different type of models

In [9]:
nan_=['Outlet_Size','Item_Weight'] 

#get the indecies of  null values
def indecies(df,l): 
    nan_col_indecies=[];nan_indecies=[]
    for i in nan_:
        s=df[df[i].isnull()].index.tolist()
        nan_col_indecies.append(s)
        nan_indecies.extend(s)
    return nan_col_indecies,nan_indecies

nan_col_indecies_tr,nan_indecies_tr = indecies(train_df,nan_)
nan_col_indecies_te,nan_indecies_te = indecies(test_df,nan_)

In [20]:
#number of null values in Outlet_Size and nan_col_indecies is 2410 as it contains null indecies
len(train_df[train_df['Outlet_Size'].isnull()]),len(nan_col_indecies_tr[0])

(2410, 2410)

In [21]:
train_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [22]:
sample_df=pd.read_csv('sample_submission_8RXa3c6.csv')
sample_df.head(10)

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1000
1,FDW14,OUT017,1000
2,NCN55,OUT010,1000
3,FDQ58,OUT017,1000
4,FDY38,OUT027,1000
5,FDH56,OUT046,1000
6,FDL48,OUT018,1000
7,FDC48,OUT027,1000
8,FDN33,OUT045,1000
9,FDA36,OUT017,1000


# Model based Imputation

In [23]:
#label encode all the categorical feature
le=LabelEncoder()
train_df[cat_columns] = train_df[cat_columns].apply(le.fit_transform)
test_df[cat_columns] = test_df[cat_columns].apply(le.fit_transform)

In [24]:
#make a copy or else both t1 and train_df might refer to same data frame 
t1=train_df.copy()
t2=test_df.copy()
t1.shape,t2.shape

#create a data frame without null values by combining train and test data
t1.drop(set(nan_indecies_tr),axis=0,inplace=True)
t2.drop(set(nan_indecies_te),axis=0,inplace=True)
t_tr=pd.concat([t1,t2])
t_tr.shape #shape concatinated dataset 

(7749, 12)

In [24]:
def model_based_prediction(col_,model_,i):
    X=t_tr[col_]
    y=t_tr[nan_[i]]
    model_= model_ 
    model_.fit(X,y) #fit model
    p1=[];p2=[]
    for j in range(len(nan_col_indecies_tr[i])):
        p=model_.predict(train_df[X.columns][nan_col_indecies_tr[i][j]:nan_col_indecies_tr[i][j]+1])#prediction of the null row
        train_df[nan_[i]][nan_col_indecies_tr[i][j]:nan_col_indecies_tr[i][j]+1]=p[0] #substitute the null with predicted value
        p1.append(p)
    for j in range(len(nan_col_indecies_te[i])):
        p=model_.predict(test_df[X.columns][nan_col_indecies_te[i][j]:nan_col_indecies_te[i][j]+1])
        test_df[nan_[i]][nan_col_indecies_te[i][j]:nan_col_indecies_te[i][j]+1]=p[0]
        p2.append(p)
    return(p1,p2)

In [25]:
col_1=['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Location_Type','Outlet_Type','Item_Visibility','Item_MRP']
p11,p12=model_based_prediction(col_1,CatBoostClassifier(silent=True),0) #pass column name and model
#return of predicted values is done to see the values and it is not necessary

col_1.append(nan_[0]) #append the predicted column for better result
p21,p22=model_based_prediction(col_1,CatBoostRegressor(silent=True),1)

Pass the features that do not have null values and type of model to fit and the function would impute the null values.In above code it is optional to append the predicted column